In [0]:
%run ./00_functions_and_libraries

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 warnings.warn(

In [0]:
%run ./01_params

Out[9]: 'if params[\'Database\'] == "prod":\n params[\'RF_LOC\'] = "X"\nelif params[\'Database\'] == "impl":\n params[\'Database\']\nelif params[\'Database\']=="val":\n params[\'Database\']\nelif params[\'db\']=="dev1":\n params[\'Database\']\nelif params[\'Database\'] == "dev0":\n params[\'RF_LOC\']\nelse:\n raise ValueError(\'{} is invalid or unmapped\'.format(db))'

In [0]:
spark.conf.set('spark.sql.execution.arrow.enabled', True)
spark.conf.set('spark.sql.execution.arrow.fallback.enabled', False)
spark.conf.set("spark.databricks.io.cache.enabled", True)
spark.conf.set("spark.sql.session.timeZone", "America/New_York")
spark.conf.set(
  params["AzureSASLocation"],
  dbutils.secrets.get(scope=params["AzureSASScope"],key=params["AzureSASKey"])
)
spark.sql('use {}'.format(params["Database"]))
intDataDir=params['intermediateDataDir']

In [0]:
''' 
  The below loop is to replicate MySQL group concat functionality that is done via sort_array and slice. 
  For a list of columns that need to be group concated through, evaluate each for a given record (group by column, MP_ID in this case),
  aggregating all of the vlues for a given column from concat list, put into a list column (via collect_list) and use sort_array to 
  sort them within the column based on the respective PCR's PM Score (in descending order). Then use concat_ws to take the first item
  from that sorted list column.
  
  df = the dataframe of interest
  group_column = the value to group by, i.e. the id column the dataframe is being rolled up to
  concat_list = a list of strings giving column names wanting to apply the group concat
  sort_column = a string of the column name to be used to sort by
  sort_order = False ; by default will sort in desc
'''  

group_column = "mp_id"
concat_list = ['last_name', 'first_name', 'middle_name', 'address', 'city', 'county', 'state', 'zip_code', 'ssn', 'gender', 'race', 'age', 'dob', 'state_issuing_drivers_license', 'drivers_license_number', 'mbi']
sort_column = "pm_score"
sort_order = False
df=spark.sql('select * from pcr_master')

df=df.fillna(100.00,subset=["pm_score"])
df_final=df.select(group_column).distinct()
df_final=df_final.withColumn('er_flag',lit(None))
for i in concat_list:
  df_helper=df.groupBy(group_column)\
  .agg(sort_array(collect_list(struct(sort_column,i)),sort_order).alias('collect_list'))\
  .withColumn("sorted_list",col("collect_list."+str(i)))\
  .withColumn("first_item",slice(col("sorted_list"),1,1))\
  .withColumn(i,concat_ws(",",col("first_item")))\
  .drop("collect_list")\
  .drop("sorted_list")\
  .drop("first_item")
  df_final=df_final.join(df_helper,group_column,"inner")
  df_final.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{intDataDir}/delta/GOLD/pcr_meta")
  df_final = spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/pcr_meta")
df_final = convertToNull(df_final)  
df_final = df_final.withColumn('er_flag', when(nonnull_count(col('last_name'), col('first_name'), col('middle_name'), col('address'), col('city'), col('county'), col('state'), col('zip_code'), col('ssn'), col('gender'), col('race'), col('age'), col('dob'), col('state_issuing_drivers_license'), col('drivers_license_number'), col('mbi')) > 10, lit(1)).otherwise(lit(0)))
df_final.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{intDataDir}/delta/GOLD/pcr_meta")